# Luftdaten data : data cleaning, resampling - mini version

# - RETRYING THIS, just to makes sure we've got the hang of it! 
## Code builds a continuous time tabular version of the luftdaen data, such that the same time period is present for each sensor in the data, regardless of whether each sensor has data for all the time slots. 

## Testing :
- using pd.resample
- constructing a time shift using pandas own tools, rather than my own


#### Reference documents

Resampling time series data with Pandas ( Ben Alex Keen ) 
http://benalexkeen.com/resampling-time-series-data-with-pandas/

Pandas reference manual : 

.at - access df values using nay kind of index, for SINGLE VALUES
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html#pandas.DataFrame.loc

.iat - only integer index values for getting/setting SINGLE df values
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iat.html

.loc - 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html#pandas.DataFrame.loc

.iloc - purely integer indexed access ( getting/setting ) values 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html#pandas.DataFrame.iloc

datetime - documentation - useful for time! 
https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

#### methods of filling … 

These are some of the common methods you might use for resampling:

Method	Description

bfill	Backward fill

count	Count of values

ffill	Forward fill

first	First valid data value

last	Last valid data value

max	Maximum data value

mean	Mean of values in time range

median	Median of values in time range

min	Minimum data value

nunique	Number of unique values

ohlc	Opening value, highest value, lowest value, closing value

pad	Same as forward fill

std	Standard deviation of values

sum	Sum of values

var	Variance of values

#### time abbreviations 

Alias	Description

B	Business day

D	Calendar day

W	Weekly

M	Month end

Q	Quarter end

A	Year end

BA	Business year end

AS	Year start

H	Hourly frequency

T, min	Minutely frequency

S	Secondly frequency

L, ms	Millisecond frequency

U, us	Microsecond frequency

N, ns	Nanosecond frequency

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
# parameters

# start_time = "2018-12-31 21:58:42"
end_time = "2019-01-01 11:58:42"
# generate this please
start_time = "?????"

time_frequency_for_periods__for_basic_data = "5T"
num_of_time_periods___for_basic_data = 24*20 # 24 hrs * 12 x 5 mins in each hour

# when generating time periods 
sampling_frequency = "3T"



# --- data urls 

curr_url = "????"
nordic_midnight_24_hrs_data__url = "/Users/miska/Documents/open_something/luftdaten/luftdaten_code/luftdaten__make_tabular_data__from_db_data/ld_NYE_midnight_24hrs_nordics_all_data_01.csv"
# nordic_midnight_24_hrs_data__url = "/home/miska/documents/opensomething/luftdaten/dustmin_to_csv__various_code/ld_NYE_midnight_24hrs_nordics_all_data_01.csv"



# set the current data source 
curr_url = nordic_midnight_24_hrs_data__url

In [3]:
# try convert the timestamp in the data, to epoch

in_data = pd.read_csv( curr_url )
in_data.shape

(127109, 7)

#### basic data checking

In [4]:
in_data.dtypes

sensor_id         int64
sensor_namee     object
lat             float64
lon             float64
timestamp        object
p1              float64
p2              float64
dtype: object

In [5]:
# is the timestamp column not an offical timestamp column?
type( in_data['timestamp'][0] )

str

In [6]:
# aha - timestamp column not a timestamp column?
# - let's fix 
in_data['timestamp'] = pd.to_datetime( in_data['timestamp'] )

In [7]:
# check the timestamps column type again
type( in_data['timestamp'][0] )

pandas._libs.tslibs.timestamps.Timestamp

In [8]:
# set the timestamp column as the index 
in_data = in_data.set_index( 'timestamp' )

In [9]:
in_data.index

DatetimeIndex(['2018-12-31 11:57:22', '2018-12-31 11:58:44',
               '2018-12-31 11:58:47', '2018-12-31 11:56:41',
               '2018-12-31 11:57:42', '2018-12-31 11:57:52',
               '2018-12-31 11:58:51', '2018-12-31 11:58:28',
               '2018-12-31 11:57:18', '2018-12-31 11:57:22',
               ...
               '2019-01-01 11:59:41', '2019-01-01 11:59:46',
               '2019-01-01 11:57:19', '2019-01-01 11:59:59',
               '2019-01-01 11:56:55', '2019-01-01 11:58:57',
               '2019-01-01 11:59:36', '2019-01-01 11:59:41',
               '2019-01-01 11:57:12', '2019-01-01 11:58:42'],
              dtype='datetime64[ns]', name='timestamp', length=127109, freq=None)

In [10]:
in_data = in_data.sort_index()

In [11]:
# check
in_data.index

DatetimeIndex(['2018-12-31 11:55:19', '2018-12-31 11:56:37',
               '2018-12-31 11:56:38', '2018-12-31 11:56:39',
               '2018-12-31 11:56:40', '2018-12-31 11:56:40',
               '2018-12-31 11:56:40', '2018-12-31 11:56:41',
               '2018-12-31 11:56:42', '2018-12-31 11:56:43',
               ...
               '2019-01-01 23:59:54', '2019-01-01 23:59:54',
               '2019-01-01 23:59:55', '2019-01-01 23:59:55',
               '2019-01-01 23:59:56', '2019-01-01 23:59:56',
               '2019-01-01 23:59:56', '2019-01-01 23:59:56',
               '2019-01-01 23:59:57', '2019-01-01 23:59:58'],
              dtype='datetime64[ns]', name='timestamp', length=127109, freq=None)

In [12]:
# order by time? 
in_data[:20]

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2018-12-31 11:55:19,13012,SDS011,57.662,12.006,6.30,2.60
2018-12-31 11:56:37,18112,SDS011,57.478,11.978,174.80,15.13
2018-12-31 11:56:38,15067,SDS011,60.024,18.770,1.62,1.02
2018-12-31 11:56:39,11765,SDS011,55.716,13.244,33.95,13.40
2018-12-31 11:56:40,14811,SDS011,57.706,11.900,63.25,10.33
2018-12-31 11:56:40,10827,SDS011,59.334,13.512,11.45,6.00
2018-12-31 11:56:40,17538,SDS011,55.612,12.972,13.68,3.12
2018-12-31 11:56:41,7406,SDS011,56.964,24.128,11.05,6.62
2018-12-31 11:56:42,16155,SDS011,59.832,17.632,1.66,1.10


In [13]:
# this works :) 
# in_data = in_data.sort_index()

In [14]:
# let's try sorting the index in a different way … just for the sake of trying
in_data = in_data.sort_values( by='timestamp' )

#### quick data printout

In [15]:
in_data.head()

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2018-12-31 11:55:19,13012,SDS011,57.662,12.006,6.30,2.60
2018-12-31 11:56:37,18112,SDS011,57.478,11.978,174.80,15.13
2018-12-31 11:56:38,15067,SDS011,60.024,18.770,1.62,1.02
2018-12-31 11:56:39,11765,SDS011,55.716,13.244,33.95,13.40
2018-12-31 11:56:40,14811,SDS011,57.706,11.900,63.25,10.33


In [16]:
in_data.tail()

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2019-01-01 23:59:56,12129,SDS011,59.340,18.040,1.85,0.80
2019-01-01 23:59:56,17538,SDS011,55.612,12.972,6.08,1.53
2019-01-01 23:59:56,12131,SDS011,59.450,17.916,1.48,0.52
2019-01-01 23:59:57,10843,SDS011,59.354,18.364,7.26,1.60
2019-01-01 23:59:58,11374,SDS011,59.258,18.008,2.65,2.01


#### quick data exploration 

In [17]:
# do a quick search of how many values there are in the first five minutes

In [18]:
first_five_mins_rows = in_data[ '2019-01-01 12:00:00' : '2019-01-01 12:05:00' ]
first_five_mins_rows.shape

(314, 6)

In [19]:
# just check how many of the sensors have entries for the first five minues
first_five_mins_rows['sensor_id'].unique().shape, in_data['sensor_id'].unique().shape

((178,), (205,))

#### now try do the data interpolation, for each sensor, so there's the same time period in each sensor

In [20]:
##### work on a smaller section of the data
in_data__smlr = in_data[::10]
in_data__smlr.shape

(12711, 6)

In [21]:
# check how it looks for how many sensors 
first_five_mins_rows__SMLR = in_data__smlr[ '2019-01-01 12:00:00' : '2019-01-01 12:05:00' ]
first_five_mins_rows__SMLR.shape, first_five_mins_rows__SMLR['sensor_id'].unique().shape

((32, 6), (30,))

###### construct the different times we could use for different indexing 

####### - eg for inserting blank rows, to get similar time periods in each sensors data
####### - eg for quering different times … eg at the beginning / end of the data series for each sensor 


In [22]:
start_time = pd.to_datetime( '2018-12-31 12:00:00' )
start_time

Timestamp('2018-12-31 12:00:00')

In [23]:
startime_plus_five_mins = start_time + pd.offsets.Minute( 5 )
startime_plus_five_mins

Timestamp('2018-12-31 12:05:00')

In [24]:
end_time = pd.to_datetime( '2019-01-01 12:00:00' )
end_time

Timestamp('2019-01-01 12:00:00')

In [25]:
end_time_minus_five_mins = end_time - pd.offsets.Minute(5 )
end_time_minus_five_mins

Timestamp('2019-01-01 11:55:00')

In [26]:
in_data_start_time = in_data[:1]

In [27]:
type( in_data_start_time )

pandas.core.frame.DataFrame

In [29]:
type( in_data_start_time.index ) #iat[ 0, ]

pandas.core.indexes.datetimes.DatetimeIndex

In [30]:
in_data_start_time.index

DatetimeIndex(['2018-12-31 11:55:19'], dtype='datetime64[ns]', name='timestamp', freq=None)

In [31]:
in_data_start_time.set_index( pd.DatetimeIndex( [ start_time ] )   )

,sensor_id,sensor_namee,lat,lon,p1,p2
2018-12-31 12:00:00,13012,SDS011,57.662,12.006,6.3,2.6


In [32]:
in_data_start_time

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2018-12-31 11:55:19,13012,SDS011,57.662,12.006,6.3,2.6


In [33]:
### make new template row for the time rows we'll insert later

In [34]:
in_data_start_time[:1]

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2018-12-31 11:55:19,13012,SDS011,57.662,12.006,6.3,2.6


In [35]:
in_data_start_time[:1]['sensor_namee'] = np.NaN
in_data_start_time[:1]['sensor_id'] = np.NaN
in_data_start_time[:1]['p1'] = np.NaN
in_data_start_time[:1]['p2'] = np.NaN
in_data_start_time[:1]['lat'] = np.NaN
in_data_start_time[:1]['lon'] = np.NaN
in_data_start_time

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2018-12-31 11:55:19,13012,SDS011,NaN,NaN,NaN,NaN


In [36]:
# NO EFFECT :-( )
in_data_start_time[:1]['sensor_namee'] = 13

In [37]:
in_data_start_time

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2018-12-31 11:55:19,13012,SDS011,NaN,NaN,NaN,NaN


### make template row for blank data, with right time, rows, to insert later
#### to make the time data for the sensor data have the same start and end 

In [38]:
in_data_templace_for_different_new_time_rows__series = in_data_start_time.iloc[0]

In [39]:
in_data_templace_for_different_new_time_rows__series

sensor_id        13012
sensor_namee    SDS011
lat                NaN
lon                NaN
p1                 NaN
p2                 NaN
Name: 2018-12-31 11:55:19, dtype: object

In [40]:
# now set the other values in the start time
in_data_templace_for_different_new_time_rows__series.at['p1'] = np.NaN
in_data_templace_for_different_new_time_rows__series.at['p2'] = np.NaN
in_data_templace_for_different_new_time_rows__series.at['lat'] = np.NaN
in_data_templace_for_different_new_time_rows__series.at['lon'] = np.NaN
in_data_templace_for_different_new_time_rows__series.at['sensor_namee'] = np.NaN
in_data_templace_for_different_new_time_rows__series.at['sensor_id'] = np.NaN
in_data_templace_for_different_new_time_rows__series

sensor_id       NaN
sensor_namee    NaN
lat             NaN
lon             NaN
p1              NaN
p2              NaN
Name: 2018-12-31 11:55:19, dtype: object

In [47]:
in_data_start_time__series = in_data_start_time
in_data_start_time__series = in_data_start_time__series.set_index( pd.DatetimeIndex( [startime_plus_five_mins] ) )
in_data_start_time__series

,sensor_id,sensor_namee,lat,lon,p1,p2
2018-12-31 12:05:00,13012,SDS011,NaN,NaN,NaN,NaN


In [54]:
type( in_data_start_time__series )

pandas.core.frame.DataFrame

In [48]:
in_data_templace_for_different_new_time_rows__series

sensor_id       NaN
sensor_namee    NaN
lat             NaN
lon             NaN
p1              NaN
p2              NaN
Name: 2018-12-31 11:55:19, dtype: object

In [49]:
start_time__blank_data_row_w_right_time = in_data_templace_for_different_new_time_rows__series

In [50]:
start_time__blank_data_row_w_right_time

sensor_id       NaN
sensor_namee    NaN
lat             NaN
lon             NaN
p1              NaN
p2              NaN
Name: 2018-12-31 11:55:19, dtype: object

#### make blank rows for start and end time 

In [51]:
start_time_blank_data_row__with_right_timestamp = in_data_templace_for_different_new_time_rows__series

In [52]:
start_time_blank_data_row__with_right_timestamp

sensor_id       NaN
sensor_namee    NaN
lat             NaN
lon             NaN
p1              NaN
p2              NaN
Name: 2018-12-31 11:55:19, dtype: object

In [67]:
start_time_blank_data_row__with_right_timestamp = pd.DataFrame( [start_time_blank_data_row__with_right_timestamp] )

In [68]:
start_time_blank_data_row__with_right_timestamp

,0
0,2018-12-31 11:55:19 sensor_id ...


In [65]:
type( start_time_blank_data_row__with_right_timestamp )

pandas.core.frame.DataFrame

In [66]:
start_time_blank_data_row__with_right_timestamp.shape

(6, 1)

In [64]:
# set starttime 
start_time_blank_data_row__with_right_timestamp.set_index( pd.DatetimeIndex( [startime_plus_five_mins] ) )

ValueError: Length mismatch: Expected axis has 6 elements, new values have 1 elements

In [116]:
start_time_blank_data_row__with_right_timestamp__as_df = pd.DataFrame( start_time_blank_data_row__with_right_timestamp ) 

In [126]:
start_time_blank_data_row__with_right_timestamp__as_df.columns = [ start_time_blank_data_row__with_right_timestamp.index ]

ValueError: Length mismatch: Expected axis has 1 elements, new values have 6 elements

In [105]:
# set_index( pd.DatetimeIndex( [ start_time ] )   )
start_time_blank_data_row__with_right_timestamp.set_index( pd.DatetimeIndex( [start_time] ))

AttributeError: 'Series' object has no attribute 'set_index'

In [67]:
# let's cheat a bit and output things in a list
out_time_complete_arrays = []

list_of_unique_sensor_ids = first_five_mins_rows__SMLR['sensor_id'].unique()

for sensor_id in list_of_unique_sensor_ids:
    
    print("--  working on sensor id "+str( sensor_id ))
    
    # check if the first and last five minues are there or not

--  working on sensor id 14017
--  working on sensor id 19597
--  working on sensor id 12127
--  working on sensor id 12131
--  working on sensor id 14264
--  working on sensor id 7597
--  working on sensor id 13443
--  working on sensor id 17018
--  working on sensor id 13307
--  working on sensor id 10843
--  working on sensor id 13423
--  working on sensor id 19599
--  working on sensor id 14807
--  working on sensor id 17020
--  working on sensor id 11546
--  working on sensor id 14837
--  working on sensor id 12385
--  working on sensor id 12389
--  working on sensor id 10922
--  working on sensor id 17532
--  working on sensor id 12393
--  working on sensor id 11767
--  working on sensor id 11404
--  working on sensor id 9436
--  working on sensor id 11006
--  working on sensor id 15067
--  working on sensor id 10281
--  working on sensor id 9411
--  working on sensor id 9368
--  working on sensor id 18110
